In [15]:
#decode part for actuators



In [ ]:
import subprocess


def read_pddl_file(file_path):
    with open(file_path, 'r') as pddl_file:
        return pddl_file.read()


class PddlPlanner:
    def __init__(self, mode='day'):
        self.mode = mode
        self.dm_day = read_pddl_file('pddl/dm_day.pddl')
        self.pb_day = read_pddl_file('pddl/pb_day.pddl')
        self.dm_night = read_pddl_file('pddl/dm_night.pddl')
        self.pb_night = read_pddl_file('pddl/pb_night.pddl')

    def generate_problem(self, init):
        with open('problem.pddl', 'w') as problem:
            if self.mode == 'day':
                problem.write(self.pb_day.strip().replace('(:init)', init))
            else:
                problem.write(self.pb_night.strip().replace('(:init)', init))

    def generate_domain(self):
        with open('domain.pddl', 'w') as domain:
            if self.mode == 'day':
                domain.write(self.dm_day.strip())
            else:
                domain.write(self.dm_night.strip())

    def get_planning_results(self, process_result):
        actions = []
        lines = str(process_result.stdout, 'utf-8').split('time spent:')[0].split('step')[1].strip().split('\n')
        for line in lines:
            actions.append(line.split(':')[1].strip().split(' ')[0].strip().lower())
        return actions

    def convert_state_to_init(self, state):
        init = "(:init"
        if (state['sensors.lightlevel'] > 60):
            init += "(TOOBRIGHT light_intensity)"
        else:
            init += "(TOODARK light_intensity)"
        if (state['sensors.pir'] == 1):
            init += "(ISATDOOR person_atdoor)"
        else:
            init += "(NOTISATDOOR person_atdoor)"
        if (state['sensors.temperature'] > 23):
            init += "(TOOHOT temperature)"
        else:
            init += "(TOOCOLD temperature)"
        if (state['sensors.ultrasonic'] < 50):
            init += "(ISPRESENT presence)"
        else:
            init += "(NOTISPRESENT presence)"
        init += ")"
        return init

    def run_planner(self, state):
        self.generate_problem(self.convert_state_to_init(state))
        self.generate_domain()
        process_result = subprocess.run(['./pddl/ff', '-o', 'domain.pddl', '-f', 'problem.pddl'], stdout=subprocess.PIPE)
        return self.get_planning_results(process_result)

In [16]:
sensors.light
sensors.pir
sensors.temperature
sensors.ultrasonic
    